In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import nltk
import sklearn
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.stem.porter import PorterStemmer #stemming is probably stupid
from nltk.corpus import wordnet
from collections import Counter, defaultdict
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package wordnet to /Users/shei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/shei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_x = pd.read_csv("./data/train_x.csv", index_col=0)
train_y = pd.read_csv("./data/train_y.csv", index_col=0)

In [3]:
def make_list(raw, sep):
    output = []
    sentence = []
    for tag in raw:
        if tag==sep:
            output.append(sentence)
            sentence = []
        sentence.append(tag)
    output.append(sentence)
    return output[1:]

In [4]:
y_train = make_list(train_y["tag"].values, "O")

In [5]:
def create_grams(y_train, n=3):
    tran_counts = defaultdict(int)
    state_counts = defaultdict(int)
    for doc in y_train:
        state = n*["O"]
        for i in range(1, len(doc)):
            if not doc[i]:
                break
            tran_counts[(tuple(state),doc[i])] += 1
            state_counts[tuple(state)] += 1
            state.pop(0)
            state.append(doc[i])
        tran_counts[(tuple(state), "END")] += 1
        state_counts[tuple(state)] += 1
    return tran_counts, state_counts

In [6]:
tran_4_counts, state_4_counts = create_grams(y_train, 4)
tran_3_counts, state_3_counts = create_grams(y_train, 3)
tran_2_counts, state_2_counts = create_grams(y_train, 2)
tran_1_counts, state_1_counts = create_grams(y_train, 1)

In [7]:
state_1_counts

defaultdict(int,
            {('O',): 1387,
             ('NNP',): 67431,
             (',',): 35475,
             ('CD',): 26323,
             ('NNS',): 43879,
             ('JJ',): 45356,
             ('MD',): 7204,
             ('VB',): 19353,
             ('DT',): 59928,
             ('NN',): 96805,
             ('IN',): 72097,
             ('.',): 28837,
             ('VBZ',): 16170,
             ('VBG',): 11013,
             ('CC',): 17297,
             ('VBD',): 21357,
             ('VBN',): 14622,
             ('RB',): 22445,
             ('TO',): 16250,
             ('PRP',): 12837,
             ('RBR',): 1257,
             ('WDT',): 3217,
             ('VBP',): 9395,
             ('RP',): 1912,
             ('PRP$',): 6065,
             ('JJS',): 1407,
             ('POS',): 6478,
             ('``',): 5229,
             ('EX',): 645,
             ("''",): 5105,
             ('WP',): 1741,
             (':',): 3602,
             ('JJR',): 2394,
             ('WRB',): 1566,
  

In [8]:
tran_counts = {1: tran_1_counts, 
               2: tran_2_counts, 
               3: tran_3_counts, 
               4: tran_4_counts}
state_counts = {1: state_1_counts, 
               2: state_2_counts, 
               3: state_3_counts, 
               4: state_4_counts}

In [9]:
emit_counts = Counter(list(zip(train_y["tag"], train_x["word"])))

In [10]:
tag_counts = Counter(train_y["tag"])

In [11]:
del tag_counts["O"] #impossible to transition to "Doc Start"

In [23]:
possible_tags = list(set([el[1] for el in tran_counts[1].keys()]))
all_tags = possible_tags + ["O"]

In [13]:
vocab_size = len(set(train_x["word"])) + 1

In [14]:
vocab_size

37506

In [15]:
dev_x = pd.read_csv("./data/test_x.csv", index_col=0)

In [16]:
x_test = make_list(dev_x["word"].values, "-DOCSTART-")

In [17]:
#transition exists 85 - 89.77% accuracy - beam search 92% accuracy
def calc_prob_backoff(state, tag):
    n = len(state)
    substate = tuple(state[:])
    while tran_counts[n][(substate, tag)] == 0:
        substate = tuple(state[-n:])
        n -= 1
        
        if n == 0: #transition from unigram doesn't exist
            s = sum([value for key, value in state_counts[1].items()]) #summing over all of state counts
            return state_counts[1][(tag,)] / s #return probability of tag existing at all

    num = tran_counts[n][substate, tag]
    den = state_counts[n][substate]
    return num/den

In [ ]:
### Beam Search
k = 1
results = []
delta = 1 # below this # of occurences we ignore, for kneser-ney smoothing
counter = 0
beam = 10
n=3
for doc in x_test:
    state = n*["O"]
    paths = {tuple(state): 0}
    if counter % 10 == 0: 
        print(counter)
    counter += 1
    for i in range(1, len(doc)):
        probs = {}
        for path, value in paths.items():
            state = path[-n:]
            for tag in possible_tags:
    ### BACKOFF ###
                prob = np.log(calc_prob_backoff(state, tag)) ##transition probability

    ### WITHOUT BACKOFF ###
    #             if(tran_counts[(tuple(state), tag)] == 0): 
    #                 prob = -10 #give some small minimimal probability **maybe play with this later?**
    #             else:
    #                 prob = np.log(tran_counts[(tuple(state), tag)]/state_counts[tuple(state)])
    #                 prob = max(prob, -10) #set a floor to the min probability

                ### K Smoothing ###
                prob += np.log((emit_counts[(tag, doc[i])] + k)/(tag_counts[tag] + k*vocab_size)) #emission probability

                ### Without K Smoothing ###
#               prob += np.log((emit_counts[(tag, doc[i])])/(tag_counts[tag]))

                probs[tuple(list(path)+[tag])] = prob + value
        
        best_n_paths = sorted([(v, i, k) for i, (k, v) in enumerate(probs.items())])[-beam:]
#         print(best_n_paths)
        paths = dict([(k, v) for v, i, k in best_n_paths])
        assert len(paths) == beam
    results.append(best_n_paths[0][2][n-1:])

In [ ]:
# Viterbi
from itertools import product
for s in product(possible_tags + ["O"], repeat=3):
    print(s)

In [ ]:
##### Viterbi:
smooth_k = 1
results = []
counter = 0
beam = 10
n=2
final_results = []
for doc in x_test:
    if counter % 1 == 0: 
        print("doc:", counter, "of ", len(x_test))
    counter += 1
    init_state = n*["O"]
    path_probs = [defaultdict(lambda: float("-inf"))]
    path_probs[-1][tuple(init_state)] = 0
    backpointer = [{tuple(init_state): None}]

    for i in range(1, len(doc)):
        if i % 100 == 0: 
            print(i," words in ", len(doc))

        prev_states = list(backpointer[-1].keys())
#         print(prev_states)
        path_probs.append(defaultdict(lambda: float("-inf")))
        backpointer.append({})
        for new_state in product(all_tags, repeat=n):
            old_to_new_probs = []
            for old_tag in all_tags: 
                old_state = tuple([old_tag] + list(new_state[:-(n-1)]))
                new_tag = new_state[-1]
                ### BACKOFF ###
                prob = np.log(calc_prob_backoff(old_state, new_tag)) ##transition probability

                ### K Smoothing ###
                prob += np.log((emit_counts[(new_tag, doc[i])] + smooth_k)/(tag_counts[new_tag] + smooth_k*vocab_size)) #emission probability
                
                prob += path_probs[-2][old_state]
                old_to_new_probs.append(prob)
#                 if old_state == ("O", "O"):
#                     print(old_state, "old_state")
#                     print(new_state, "new_state")
#                     print(new_tag, "new_tag")
#                     print(calc_prob_backoff(old_state, new_tag), 'state')
#                     print(prob, "prob")
            index = np.argmax(old_to_new_probs)
            path_probs[-1][new_state] = old_to_new_probs[index]
            backpointer[-1][new_state] = tuple([all_tags[index]] + list(new_state[:-(n-1)]))
    state = max(path_probs[-1], key=path_probs[-1].get)
    results = []
    results.append(state[-1])
    for bp in list(reversed(backpointer))[:-1]:
        state = bp[state]
        results.append(state[-1])
    final_results.append(list(reversed(results)))

In [36]:
flatten_results = np.concatenate(final_results)

In [37]:
results_data=np.concatenate([np.arange(len(flatten_results)).reshape(-1,1),np.asarray(flatten_results).reshape(-1,1)], axis=1)
results_df = pd.DataFrame(data=results_data, columns=["id", "tag"])
results_df

,id,tag
0,0,O
1,1,NNP
2,2,MD
3,3,VB
4,4,JJ
...,...,...
236577,236577,PRP
236578,236578,RB
236579,236579,IN
236580,236580,PRP


In [38]:
results_df.to_csv("./results/test_y.csv", index=False)

In [39]:
flatten_results

array(['O', 'NNP', 'MD', ..., 'IN', 'PRP', '.'], dtype='<U4')

In [41]:
len(x_test)

463

In [ ]:
np.save('final_results', final_resulst)